In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [79]:
df_temp_country=pd.read_csv('/content/drive/MyDrive/Spatial Data /Copy of GlobalLandTemperaturesByCountry.csv')

In [80]:
df_temp_country.shape

(577462, 4)

In [81]:
df_temp_country.isnull().sum()

,0
dt,0
AverageTemperature,32651
AverageTemperatureUncertainty,31912
Country,0


In [82]:
df_temp_country.dropna(subset=['AverageTemperature'],inplace=True)

In [83]:
df_temp_country.isnull().sum()

,0
dt,0
AverageTemperature,0
AverageTemperatureUncertainty,0
Country,0


In [84]:
df_temp_country['Country'].unique()

array(['Åland', 'Afghanistan', 'Africa', 'Albania', 'Algeria',
       'American Samoa', 'Andorra', 'Angola', 'Anguilla',
       'Antigua And Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Asia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Baker Island', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium',
       'Belize', 'Benin', 'Bhutan', 'Bolivia',
       'Bonaire, Saint Eustatius And Saba', 'Bosnia And Herzegovina',
       'Botswana', 'Brazil', 'British Virgin Islands', 'Bulgaria',
       'Burkina Faso', 'Burma', 'Burundi', "Côte D'Ivoire", 'Cambodia',
       'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Chile', 'China',
       'Christmas Island', 'Colombia', 'Comoros',
       'Congo (Democratic Republic Of The)', 'Congo', 'Costa Rica',
       'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic',
       'Denmark (Europe)', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt'

In [85]:
replacement_dict = {
    'Denmark (Europe)': 'Denmark',
    'France (Europe)': 'France',
    'Netherlands (Europe)': 'Netherlands',
    'United Kingdom (Europe)': 'United Kingdom'
    # Add more replacements if needed
}

# Replace duplicate names in the DataFrame
df_temp_country['Country'] = df_temp_country['Country'].replace(replacement_dict)


In [86]:
df_temp_country['Country'].nunique()

238

# What Countrywise Avg Temperature


In [87]:
avg_temp=df_temp_country.groupby('Country',as_index=False).agg({'AverageTemperature':'mean'}).sort_values(by='AverageTemperature',ascending=False)

In [88]:
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [89]:
import plotly.express as px
import plotly.io as pio

# Use 'colab' renderer for displaying the map in Google Colab
pio.renderers.default = 'colab'

# Create the choropleth map
fig = px.choropleth(
    avg_temp,
    locations='Country',
    locationmode='country names',
    color='AverageTemperature',
    color_continuous_scale=px.colors.sequential.Viridis,  # Optional: change the color scale
    title='Average Temperature in Countries'
)

# Update layout for better visualization
fig.update_layout(
    title_text='Average Temperature in Countries'
)

# Show the map
fig.show()


We can use above chart to see average country temperature

# Is there Global Warming


In [90]:
df_global=pd.read_csv('/content/drive/MyDrive/Spatial Data /Copy of GlobalTemperatures.csv')

In [91]:
df_global.head()

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty
0,1750-01-01,3.034,3.574,NaN,NaN,NaN,NaN,NaN,NaN
1,1750-02-01,3.083,3.702,NaN,NaN,NaN,NaN,NaN,NaN
2,1750-03-01,5.626,3.076,NaN,NaN,NaN,NaN,NaN,NaN
3,1750-04-01,8.490,2.451,NaN,NaN,NaN,NaN,NaN,NaN
4,1750-05-01,11.573,2.072,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
def fetch_date(date):
    return date.split('-')[0]

In [93]:
df_global['Year']=df_global['dt'].apply(fetch_date)

In [94]:
df_global_yearavg= df_global.groupby('Year',as_index=False).agg({'LandAverageTemperature':'mean','LandAverageTemperatureUncertainty':'mean'})

In [95]:
df_global_yearavg

,Year,LandAverageTemperature,LandAverageTemperatureUncertainty
0,1750,8.719364,2.637818
1,1751,7.976143,2.781143
2,1752,5.779833,2.977000
3,1753,8.388083,3.176000
4,1754,8.469333,3.494250
...,...,...,...
261,2011,9.516000,0.082000
262,2012,9.507333,0.083417
263,2013,9.606500,0.097667
264,2014,9.570667,0.090167


In [96]:
df_global_yearavg['Uncertainity Top']=df_global_yearavg['LandAverageTemperature']+df_global_yearavg['LandAverageTemperatureUncertainty']
df_global_yearavg['Uncertainity Bottom']=df_global_yearavg['LandAverageTemperature']-df_global_yearavg['LandAverageTemperatureUncertainty']

In [97]:
px.line(df_global_yearavg,x='Year',y=['LandAverageTemperature','Uncertainity Top','Uncertainity Bottom'],title='Global Warming')

As we can see there is increase in global average temperature.

# Average Temperature in each Season

In [98]:
df_global['dt']=pd.to_datetime(df_global['dt'])

In [99]:
df_global['Month']=df_global['dt'].dt.month

In [100]:
def get_season(month):
  if month>3 and month<=5:
    return 'Spring'
  elif month>=6 and month<=8:
    return 'Summer'
  elif month>=9 and month<=11:
    return 'Autumn'
  else:
    return 'Winter'



In [101]:
df_global['Season']=df_global['Month'].apply(get_season)

In [102]:
season_temp_df = df_global.pivot_table(
    values='LandAverageTemperature',
    index='Year',
    columns='Season',
    aggfunc='mean'
)



In [103]:
season_temp_df

Season,Autumn,Spring,Summer,Winter
Year,,,,
1750,8.890000,10.0315,14.518333,3.628750
1751,10.673000,7.6700,14.116000,3.086000
1752,7.587000,8.2650,NaN,3.746667
1753,9.212333,10.2290,14.608333,2.184250
1754,8.957333,11.7730,14.208333,2.147250
...,...,...,...,...
2011,10.026000,10.7345,14.954667,4.445250
2012,10.208000,11.1330,14.762667,4.227500
2013,10.278000,10.6195,14.771000,4.723000


In [104]:
px.line(season_temp_df,title='Average Temperature in each Season',
        y=['Spring','Summer','Autumn','Winter'],
        )

# Average Temperatur of USA States


In [105]:
df_states=pd.read_csv('/content/drive/MyDrive/Spatial Data /Copy of GlobalLandTemperaturesByState.csv')

In [106]:
df_states.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
0,1855-05-01,25.544,1.171,Acre,Brazil
1,1855-06-01,24.228,1.103,Acre,Brazil
2,1855-07-01,24.371,1.044,Acre,Brazil
3,1855-08-01,25.427,1.073,Acre,Brazil
4,1855-09-01,25.675,1.014,Acre,Brazil


In [107]:
df_usa=df_states[df_states['Country']=='United States']

In [108]:
df_usa.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
7458,1743-11-01,10.722,2.898,Alabama,United States
7459,1743-12-01,NaN,NaN,Alabama,United States
7460,1744-01-01,NaN,NaN,Alabama,United States
7461,1744-02-01,NaN,NaN,Alabama,United States
7462,1744-03-01,NaN,NaN,Alabama,United States


In [109]:
df_usa.dropna(inplace=True)

<ipython-input-109-f75f4bc71c9e>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [110]:
df_usa.isnull().sum()

,0
dt,0
AverageTemperature,0
AverageTemperatureUncertainty,0
State,0
Country,0


In [111]:
df_usa['State'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District Of Columbia',
       'Florida', 'Georgia (State)', 'Hawaii', 'Idaho', 'Illinois',
       'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
       'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [112]:
df_usa['State'] = df_usa['State'].replace({
    'Georgia (State)': 'Georgia'
})


<ipython-input-112-ecd11d112f7c>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [113]:
df_usa['State'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District Of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [114]:
usa=df_usa.groupby('State',as_index=False).agg({'AverageTemperature':'mean'})

In [115]:
usa.head()

,State,AverageTemperature
0,Alabama,17.066138
1,Alaska,-4.890738
2,Arizona,15.381526
3,Arkansas,15.573963
4,California,14.327677


In [116]:
!pip install opencage

In [117]:
API='54ee15ac8bf544de8c7601289dc1d577'
from opencage.geocoder import OpenCageGeocode
geocoder=OpenCageGeocode(API)

In [118]:
usa['Latitude']=None
usa['Longitude']=None
for i,row in usa.iterrows():
  query=row['State']
  result=geocoder.geocode(query)
  if len(result)>0:
    lat=result[0]['geometry']['lat']
    lng=result[0]['geometry']['lng']
    usa.at[i,'Latitude']=lat
    usa.at[i,'Longitude']=lng

In [119]:
usa.head()

,State,AverageTemperature,Latitude,Longitude
0,Alabama,17.066138,33.258882,-86.829534
1,Alaska,-4.890738,64.445961,-149.680909
2,Arizona,15.381526,34.395342,-111.763275
3,Arkansas,15.573963,35.204888,-92.447911
4,California,14.327677,36.701463,-118.755997


In [120]:
import folium
from folium.plugins import HeatMap
basemap=folium.Map()
HeatMap(data=usa[['Latitude','Longitude','AverageTemperature']]).add_to(basemap)
# Add a title to the map
title_html = '''
     <h3 align="center" style="font-size:20px"><b>Average USA States Temperature</b></h3>
     '''
basemap.get_root().html.add_child(folium.Element(title_html))

basemap

# Average Monthly Temperature of Indian Cities

In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [122]:
cities=pd.read_csv('/content/drive/MyDrive/Spatial Data /Copy of GlobalLandTemperaturesByCity.csv')

In [123]:
cities.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [124]:
India=cities[cities['Country']=='India']
India.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
49880,1816-03-01,19.934,2.258,Abohar,India,29.74N,73.85E
49881,1816-04-01,26.641,3.398,Abohar,India,29.74N,73.85E
49882,1816-05-01,32.535,2.408,Abohar,India,29.74N,73.85E
49883,1816-06-01,33.254,2.123,Abohar,India,29.74N,73.85E
49884,1816-07-01,31.105,1.848,Abohar,India,29.74N,73.85E


In [125]:
print(India.shape)
print(India.isnull().sum())
print(India['City'].nunique())

(1014906, 7)
dt                                   0
AverageTemperature               54358
AverageTemperatureUncertainty    54358
City                                 0
Country                              0
Latitude                             0
Longitude                            0
dtype: int64
391


In [126]:
India.dropna(inplace=True)

<ipython-input-126-4bc3fa32abde>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [127]:
India.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
49880,1816-03-01,19.934,2.258,Abohar,India,29.74N,73.85E
49881,1816-04-01,26.641,3.398,Abohar,India,29.74N,73.85E
49882,1816-05-01,32.535,2.408,Abohar,India,29.74N,73.85E
49883,1816-06-01,33.254,2.123,Abohar,India,29.74N,73.85E
49884,1816-07-01,31.105,1.848,Abohar,India,29.74N,73.85E


In [128]:
India['Latitude']=India['Latitude'].str.strip('N')
India['Longitude']=India['Longitude'].str.strip('E')

<ipython-input-128-f06d48a282c2>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-128-f06d48a282c2>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [129]:
metro_city=['Bombay','New Delhi','Madras','Varanasi','Bangalore','Pune','Gurgaon','Varanasi']

In [130]:
India=India[India['City'].isin(metro_city)]
India.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
630113,1796-01-01,22.672,2.317,Bangalore,India,12.05,77.26
630114,1796-02-01,24.420,1.419,Bangalore,India,12.05,77.26
630115,1796-03-01,26.092,2.459,Bangalore,India,12.05,77.26
630116,1796-04-01,27.687,1.746,Bangalore,India,12.05,77.26
630117,1796-05-01,27.619,1.277,Bangalore,India,12.05,77.26


In [131]:
coord=India.groupby('City',as_index=False).agg({'Latitude':'first','Longitude':'first'})

In [132]:
India['dt']=pd.to_datetime(India['dt'])
India['Month']=India['dt'].dt.month
India['Year']=India['dt'].dt.year
India.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,Month,Year
630113,1796-01-01,22.672,2.317,Bangalore,India,12.05,77.26,1,1796
630114,1796-02-01,24.420,1.419,Bangalore,India,12.05,77.26,2,1796
630115,1796-03-01,26.092,2.459,Bangalore,India,12.05,77.26,3,1796
630116,1796-04-01,27.687,1.746,Bangalore,India,12.05,77.26,4,1796
630117,1796-05-01,27.619,1.277,Bangalore,India,12.05,77.26,5,1796


In [133]:
monthly_temp=India.groupby(['City','Month'],as_index=False).agg({'AverageTemperature':'mean'})
monthly_temp.head()

,City,Month,AverageTemperature
0,Bangalore,1,22.713981
1,Bangalore,2,24.656619
2,Bangalore,3,27.062186
3,Bangalore,4,27.988517
4,Bangalore,5,27.522754


In [134]:
monthly_temp=monthly_temp.merge(coord,on='City',how='left')

In [135]:
monthly_temp.head()

,City,Month,AverageTemperature,Latitude,Longitude
0,Bangalore,1,22.713981,12.05,77.26
1,Bangalore,2,24.656619,12.05,77.26
2,Bangalore,3,27.062186,12.05,77.26
3,Bangalore,4,27.988517,12.05,77.26
4,Bangalore,5,27.522754,12.05,77.26


In [136]:
import folium
from folium import plugins
from folium.plugins import HeatMapWithTime

# Create a base map
india_map = folium.Map(location=[20.5937, 78.9629], zoom_start=5)

# Prepare data for HeatMapWithTime
data = []
for month in range(1, 13):
    month_data = monthly_temp[monthly_temp['Month'] == month][['Latitude', 'Longitude', 'AverageTemperature']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist()
    data.append(month_data)

# Create HeatMapWithTime and add it to the map
HeatMapWithTime(
    data,
    radius=15,
    gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'},
    min_opacity=0.5,
    max_opacity=0.8,
    use_local_extrema=True
).add_to(india_map)

# Add a title to the map
title_html = '''
    <h3 align="center" style="font-size:20px"><b>Average Monthly Temperature of Indian Metro Cities</b></h3>
    '''
india_map.get_root().html.add_child(folium.Element(title_html))

# Display the map
india_map
